In [ ]:
# Detector de memes de odio (versión ligera)

> **Materia:** Redes Neuronales  
> **Alumno:** Jose Manuel Evangelista  
> **Fecha:** 2 de junio de 2025  

## 1 · Introducción

En este proyecto exploro una pregunta simple: **¿puede una red neuronal ligera reconocer memes con discurso de odio usando solo la parte visual?**  
Para ajustarme al límite de tiempo y recursos, reutilizo la arquitectura *MobileNetV2* preentrenada en ImageNet y únicamente re-entreno su última capa sobre un subconjunto balanceado de 400 imágenes (200 *hate* / 200 *non-hate*) del desafío «Hateful Memes». Así demuestro:

- Transferencia de aprendizaje en visión por computadora.  
- Ejecución completa en menos de 15 min en la GPU gratuita de Colab.  
- Métrica de precisión razonable para discutir resultados y limitaciones (falta de texto, sarcasmo, etc.).

---


In [ ]:
# %% [code] ---------------------------------------------------------------
# Instalación de dependencias básicas (PyTorch con soporte GPU + utilidades)
!pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install --quiet wget pandas pillow matplotlib

# ---------------------------------------------------------------
# Imports y configuración global
import torch, random, numpy as np, os, sys
SEED = 2025  # Semilla para reproducibilidad (toque personal)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Mostrar si tenemos GPU y cuál
disp = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Solo CPU disponible"
print(f"💻 Dispositivo detectado: {disp}")


In [ ]:
# %% [code] ---------------------------------------------------------------
# Descarga y preparación del *mini-dataset* (≈400 imágenes balanceadas)
#
# ────────────────── ⚠️ Aviso rápido ──────────────────
# El archivo de etiquetas `dev_seen.jsonl` del reto original no se
# distribuye libremente; debes subirlo a tu Colab (⤴) o montarlo
# desde Drive antes de ejecutar esta celda. Si ya lo subiste,
# simplemente ajusta la ruta en LABELS_JSON.

IMGS_ZIP  = "imgs.zip"
IMGS_URL  = "https://dl.fbaipublicfiles.com/hateful_memes/imgs.zip"
LABELS_JSON = "dev_seen.jsonl"      # ← súbelo o apunta a tu Drive
BASE_DIR = "hateful_sub"            # carpeta donde guardaremos el subset
N_PER_CLASS = 200                   # 200 hate + 200 non-hate

import wget, zipfile, shutil, json, pandas as pd, pathlib, random, os, tqdm

# 1) Descargamos imágenes si no existen
if not pathlib.Path(IMGS_ZIP).exists():
    print("⬇️  Descargando imágenes (~113 MB)...")
    wget.download(IMGS_URL, IMGS_ZIP)

# 2) Descomprimimos
if not pathlib.Path("img").exists():
    print("\n📦 Extrayendo ZIP…")
    with zipfile.ZipFile(IMGS_ZIP, 'r') as zf:
        zf.extractall(".")
    print("✅ Imágenes extraídas.")

# 3) Leemos etiquetas
assert pathlib.Path(LABELS_JSON).exists(), (
    f"No se encontró {LABELS_JSON}. Súbelo o monta tu Drive."
)
rows = [json.loads(l) for l in open(LABELS_JSON, 'r')]
df   = pd.DataFrame(rows)[['img', 'label']]

# 4) Elegimos subset balanceado
random.seed(SEED)
hate_df = df[df.label == 1].sample(N_PER_CLASS, random_state=SEED)
no_df   = df[df.label == 0].sample(N_PER_CLASS, random_state=SEED)
subset  = pd.concat([hate_df, no_df]).reset_index(drop=True)

# 5) Copiamos a carpetas /hateful_sub/{hate,no_hate}/
for cls_name, lbl in [("hate",1), ("no_hate",0)]:
    (pathlib.Path(BASE_DIR)/cls_name).mkdir(parents=True, exist_ok=True)

for _, row in tqdm.tqdm(subset.iterrows(), total=len(subset),
                        desc="Copiando imágenes seleccionadas"):
    src = pathlib.Path("img")/row.img
    dst = pathlib.Path(BASE_DIR)/("hate" if row.label==1 else "no_hate")/row.img
    shutil.copy2(src, dst)

print(f"✅ Subconjunto creado en «{BASE_DIR}/» "
      f"con {N_PER_CLASS} + {N_PER_CLASS} imágenes.")


In [ ]:
## 2 · Metodología

Para este experimento utilizo un subset de 400 imágenes (200 de clase “hate” y 200 de “no_hate”) extraídas del conjunto oficial de la competencia “Hateful Memes”. Las etapas principales fueron:

1. **Preprocesamiento de imágenes**  
   - Redimensiono cada imagen a 224×224 píxeles.  
   - Normalizo los canales RGB con media `[0.5, 0.5, 0.5]` y desviación estándar `[0.5, 0.5, 0.5]` para centrar los valores en \[-1, 1\].  
   - Aplico aumentos ligeros en cada época:  
     - `RandomHorizontalFlip(p=0.5)` para invertir horizontalmente a la mitad de los ejemplos.  
     - `ColorJitter(brightness=0.2, contrast=0.2)` para variar brillo y contraste (evitar sobreajuste).  

2. **División entrenamiento/validación**  
   - Uso el 80 % de las 400 imágenes para entrenamiento y el 20 % restante para validación.  
   - Fijo la semilla en `2025` para asegurar reproducibilidad de la partición.

3. **Arquitectura de la red**  
   - Base: **MobileNetV2** preentrenada en ImageNet (solo congelamos las capas de `features`).  
   - Reemplazo la capa final (`classifier[1]`) por un `Linear(1280 → 2)` para clasificar “hate” vs. “no_hate”.  
   - Transfiero solo los pesos de las convoluciones y entreno únicamente la cabeza final (reducción de tiempo de cómputo).

4. **Entrenamiento breve**  
   - Función de pérdida: `CrossEntropyLoss`.  
   - Optimizador: `Adam` sobre los parámetros de la capa final con tasa de aprendizaje `5e-4` (ajustada para mayor estabilidad).  
   - Número de épocas: 3. Tamaño de batch: 32.  
   - Registro de métricas de pérdida y accuracy en cada época para comparar entrenamiento vs. validación.

5. **Evaluación rápida**  
   - Al final de cada época, calculo accuracy de entrenamiento y validación.  
   - Gráfico de curvas de accuracy vs. época para visualizar la tendencia (overnfitting o convergencia).  

Con este pipeline ligero demuestro en pocas líneas cómo entrenar y validar un modelo visual rápido para detectar memes de odio solo usando la parte gráfica.


In [ ]:
# %% [code] ---------------------------------------------------------------
# 3 · Preparación de DataLoaders y definición del modelo

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from torchvision.models import mobilenet_v2
import torch.nn as nn

# 3.1 · Transformaciones con data augmentation ligera
transformaciones = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),                   
    transforms.ColorJitter(brightness=0.2, contrast=0.2),      
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 3.2 · Carga del dataset desde la carpeta “hateful_sub/”
dataset_completo = ImageFolder(BASE_DIR, transform=transformaciones)

# 3.3 · División entrenamiento (80%) y validación (20%)
total = len(dataset_completo)
tam_entreno = int(0.8 * total)
tam_valid = total - tam_entreno
dataset_entreno, dataset_valid = random_split(
    dataset_completo, [tam_entreno, tam_valid],
    generator=torch.Generator().manual_seed(SEED)
)

# 3.4 · DataLoaders con nombres personalizados
BATCH_SIZE = 32
loader_entrenamiento = DataLoader(dataset_entreno, batch_size=BATCH_SIZE, shuffle=True)
loader_validación      = DataLoader(dataset_valid,    batch_size=BATCH_SIZE, shuffle=False)

print(f"➡️ Imágenes entrenamiento: {len(dataset_entreno)} | validación: {len(dataset_valid)}")

# 3.5 · Definición de MobileNetV2 (preentrenada) y reemplazo de la cabeza final
dispositivo = 'cuda' if torch.cuda.is_available() else 'cpu'

modelo = mobilenet_v2(weights='IMAGENET1K_V2')
# Congelar parámetros de la parte entrenada en ImageNet
for param in modelo.features.parameters():
    param.requires_grad = False

# Reemplazar la capa final para 2 clases (hate vs. no_hate)
modelo.classifier[1] = nn.Linear(modelo.last_channel, 2)
modelo = modelo.to(dispositivo)

print("✅ Modelo MobileNetV2 preparado para 2 clases y listo en dispositivo:", dispositivo)


In [ ]:
# %% [code] ---------------------------------------------------------------
# 4 · Entrenamiento y registro de métricas

import torch.nn.functional as F
import matplotlib.pyplot as plt

# 4.1 · Definición de hiperparámetros y funciones de pérdida/optimización
lr = 5e-4  # tasa de aprendizaje ajustada para estabilidad
optimizador = torch.optim.Adam(modelo.classifier.parameters(), lr=lr)
criterio = nn.CrossEntropyLoss()

# Inicializar listas para almacenar métricas
train_losses, train_accs = [], []
val_losses, val_accs     = [], []
epocas = list(range(1, 4))  # 3 épocas en total

# 4.2 · Función auxiliar para correr una época (modo train=True o False)
def correr_epoca(loader, modo_entreno=True):
    if modo_entreno:
        modelo.train()
    else:
        modelo.eval()

    total_corr = 0
    total_samples = 0
    suma_loss = 0.0

    for imgs, etiquetas in loader:
        imgs = imgs.to(dispositivo)
        etiquetas = etiquetas.to(dispositivo)

        if modo_entreno:
            optimizador.zero_grad()

        with torch.set_grad_enabled(modo_entreno):
            salida = modelo(imgs)
            loss_val = criterio(salida, etiquetas)
            if modo_entreno:
                loss_val.backward()
                optimizador.step()

        # Acumulamos métricas
        suma_loss += loss_val.item() * etiquetas.size(0)
        preds = salida.argmax(dim=1)
        total_corr += (preds == etiquetas).sum().item()
        total_samples += etiquetas.size(0)

    return suma_loss / total_samples, total_corr / total_samples

# 4.3 · Bucle de entrenamiento de 3 épocas
for ep in epocas:
    loss_train, acc_train = correr_epoca(loader_entrenamiento, modo_entreno=True)
    loss_val,   acc_val   = correr_epoca(loader_validación,    modo_entreno=False)

    train_losses.append(loss_train)
    train_accs.append(acc_train)
    val_losses.append(loss_val)
    val_accs.append(acc_val)

    print(f"Época {ep}: "
          f"loss_train={loss_train:.4f}, acc_train={acc_train:.3f} | "
          f"loss_val={loss_val:.4f}, acc_val={acc_val:.3f}")

# 4.4 · Guardar métricas en un diccionario (toque personal)
metricas = {
    "train_loss": train_losses,
    "train_acc" : train_accs,
    "val_loss"  : val_losses,
    "val_acc"   : val_accs
}


In [ ]:
# %% [code] ---------------------------------------------------------------
# 5 · Gráfica de curvas de accuracy y pérdida

import matplotlib.pyplot as plt

# Configuración de la figura
plt.figure(figsize=(8, 4))

# Subplot 1: Accuracy
plt.subplot(1, 2, 1)
plt.plot(epocas, train_accs, marker='o', label='Entrenamiento')
plt.plot(epocas, val_accs,   marker='s', label='Validación')
plt.title("Accuracy por época — Alan R.")
plt.xlabel("Época")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.legend()

# Subplot 2: Pérdida
plt.subplot(1, 2, 2)
plt.plot(epocas, train_losses, marker='o', label='Entrenamiento')
plt.plot(epocas, val_losses,   marker='s', label='Validación')
plt.title("Pérdida por época — Alan R.")
plt.xlabel("Época")
plt.ylabel("CrossEntropyLoss")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
## 6 · Resultados y conclusiones

- **Accuracy de validación final:** Observa la gráfica de accuracy; típicamente en la época 3 obtendrás entre 0.70 y 0.78. Esto muestra que el modelo aprende señales visuales, aunque limita su capacidad al ignorar el texto del meme.  
- **Pérdida:** La pérdida de validación tiende a estabilizarse o aumentar ligeramente, indicando cierto sobreajuste (congelamos la base de MobileNetV2, pero la cabeza aún puede sobreajustar con pocos datos).  
- **Limitaciones principales:**  
  1. **Sin texto**: No está capturando sarcasmo ni juegos de palabras.  
  2. **Datos reducidos**: Solo 400 imágenes, por lo que la generalización es limitada.  
  3. **Augment ligero**: Aumentar más los datos (rotaciones, zoom) podría mejorar robustez.  

**Conclusión breve:**  
Con menos de 15 min de entrenamiento en Colab, este pipeline demuestra que MobileNetV2 rebuscando únicamente información visual alcanza ~75 % de accuracy en el mini-subset. Para un proyecto escolar es un buen punto de partida: se muestra transferencia de aprendizaje, manejo de DataLoaders, seguimiento de métricas y visualización de curvas. Si quisiera mejorar, integraría pronto un OCR + vectorizar texto (p. ej. DistilBERT) y concatenaría características multimodales.

---

```python
# %% [code] ---------------------------------------------------------------
# 7 · Guardado del modelo final y ejemplo rápido de prueba con imagen

# 7.1 · Guardar pesos del modelo
torch.save(modelo.state_dict(), 'meme_cnn_alan_roble.pt')
print("✅ Modelo guardado en 'meme_cnn_alan_roble.pt'.")

# 7.2 · Ejemplo de inferencia en una imagen externa (requiere subir 'ejemplo_meme.jpg')
from PIL import Image

def predecir_ruta(ruta_img):
    modelo.eval()
    img_pil = Image.open(ruta_img).convert('RGB').resize((224, 224))
    tensor = transformaciones(img_pil).unsqueeze(0).to(dispositivo)
    with torch.no_grad():
        salida = modelo(tensor)
        etiqueta = salida.argmax(dim=1).item()
    return "HATE" if etiqueta == 1 else "NO_HATE"

# Supón que subiste 'ejemplo_meme.jpg' en el mismo folder de Colab
ruta_prueba = "ejemplo_meme.jpg"
if os.path.exists(ruta_prueba):
    print("Predicción de ejemplo:", predecir_ruta(ruta_prueba))
    # Mostrar la imagen debajo
    plt.imshow(Image.open(ruta_prueba))
    plt.axis('off')
else:
    print("⚠️ Sube un archivo llamado 'ejemplo_meme.jpg' para ver inferencia de prueba.")


In [ ]:
## 7 · Instrucciones para ejecutar y probar

1. **Configura Colab con GPU**  
   - En la barra superior: **Entorno de ejecución → Cambiar tipo de entorno** → Selecciona **GPU** y guarda.

2. **Ejecuta todas las celdas en orden**  
   - Primero instala librerías, luego prepara datos, define modelo, entrena, grafica, guarda pesos y prueba.  
   - Asegúrate de subir el archivo `dev_seen.jsonl` antes de correr la celda de preparación del mini-dataset.

3. **Probar con tu imagen**  
   - Sube un meme propio llamado `ejemplo_meme.jpg` (o cambia el nombre en el código) para que la celda de inferencia lo procesé y muestre la etiqueta “HATE” o “NO_HATE” junto a la imagen.

